In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import time
import re
from datetime import datetime, timedelta
import pymysql
from selenium.webdriver.support.ui import WebDriverWait

In [2]:
url="https://www.amazon.com/-/zh_TW/s?i=electronics-intl-ship&bbn=16225009011&rh=n%3A541966%2Cn%3A193870011%2Cn%3A17923671011%2Cn%3A284822&dc&page={}&language=zh_TW&qid=1598425548&rnid=17923671011&ref=sr_pg_{}"
options = webdriver.ChromeOptions()
options.add_argument("--disable-notifications")
options.add_argument('headless')
chrome = webdriver.Chrome('./chromedriver',chrome_options=options)
data = {}
tStart = time.time()
for i in range(1,10):
    chrome.get(url.format(i,i))
    try:
        soup = BeautifulSoup(chrome.page_source,'lxml')
        amazon_card = soup.find_all(class_="a-size-medium a-color-base a-text-normal")
        amazon_price = soup.find_all(class_="a-price-whole")   
        data.setdefault('card',[]).append([ele.text for ele in amazon_card]) 
        data.setdefault('price',[]).append([temp01.text for temp01 in amazon_price])
    except Exception as ex:
        print(ex)
chrome.close() 
tEnd = time.time()
print("總共耗時:%f 秒" % (tEnd-tStart))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


總共耗時:19.957761 秒


In [3]:
# 資料庫設定
db_settings = {
    "host": "127.0.0.1",
    "port": 3307,
    "user": "root",
    "password": "0531",
    "db": "python",
    "charset": "utf8"
}

In [17]:
search_date = datetime.strftime(datetime.now() , '%Y-%m-%d')
try:
    # 建立Connection物件
    conn = pymysql.connect(**db_settings)
    # 建立Cursor物件
    with conn.cursor() as cursor:
        for adx,datas in enumerate(data['card']):
            for every_data in datas:                
                matchObj = re.split(r'\s', every_data, re.M|re.I)
                insert = "INSERT INTO `price_comparison` (`brand`, `brand_item`, `page`, `search_date`)"
                insert += " VALUES ( %s, %s,%s,%s);"
                cursor.execute(insert,(matchObj[0],every_data,adx+1,search_date))
        conn.commit() 
        conn.close()
        cursor.close()
except Exception as ex:
    print(ex)

In [21]:
for adx,datas in enumerate(data['card']):
    
    for love,every_data in enumerate(datas):
        print(data['price'][adx][love])
        matchObj = re.split(r'\s', every_data, re.M|re.I)
        

209.
239.
209.
159.
51.
88.


IndexError: list index out of range

In [47]:
insert = "INSERT INTO `price_comparison` (`brand`, `brand_item`, `page`, `search_date`)"
                insert += " VALUES ( %s, %s, %i,%s);"
                cursor.execute(insert,matchObj[0],every_data,adx+1,search_date)
        conn.commit() 

<class 'str'>


In [28]:
conn.close()
cursor.close()